### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [5]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

In [6]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    tm = str(today - relativedelta(days=1))
    lm = str((today - relativedelta(months=6)).replace(day=1))
else:
    tm = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))
    lm = str((today - relativedelta(months=5)).replace(day=1))

### QUERIES

In [7]:
q_orders = '''SELECT cn.country_name AS Country,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       o.business_type.business_type_name AS Business,
       COUNT(DISTINCT o.order_id) AS Total,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.order_id ELSE NULL END) AS Logistic,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('REJECTED','PENDING','CANCELLED') THEN o.order_id ELSE NULL END) AS FR_Num,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('REJECTED','PENDING','CANCELLED') AND o.fail_rate_owner = 'PedidosYa' THEN o.order_id ELSE NULL END) AS PFR_Num,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('REJECTED','PENDING','CANCELLED') AND o.fail_rate_owner = 'Restaurant' THEN o.order_id ELSE NULL END) AS VFR_Num,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('REJECTED','PENDING','CANCELLED') AND o.fail_rate_owner = 'User' THEN o.order_id ELSE NULL END) AS UFR_Num,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('REJECTED','PENDING','CANCELLED') AND o.fail_rate_owner = 'Rider' THEN o.order_id ELSE NULL END) AS RFR_Num,
       COUNT(DISTINCT o.user.id) AS Total_Users,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.user.id ELSE NULL END) AS Active_Users,
       COUNT(DISTINCT CASE WHEN o.shipping_amount_no_discount = 0 AND o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS DF_Free,
       COUNT(DISTINCT CASE WHEN o.shipping_amount_no_discount = 0 AND o.with_logistics AND o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS DF_Free_Log,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN o.commission_amount / ce.rate_eu ELSE NULL END) AS Confirmed_Revenue,
       SUM(CASE WHEN o.with_logistics AND o.order_status = 'CONFIRMED' THEN o.commission_amount / ce.rate_eu ELSE 0 END) AS Confirmed_Revenue_Log,
       COUNT(DISTINCT CASE WHEN o.has_voucher_discount > 0 AND o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed_Vouchers,
       COUNT(DISTINCT CASE WHEN o.has_joker_discount > 0 AND o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed_Jokers,
       COUNT(DISTINCT CASE WHEN o.has_joker_discount > 0 AND o.with_logistics AND o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed_Jokers_Log,
       SUM(CASE WHEN o.with_logistics AND o.order_status = 'CONFIRMED' THEN (o.total_amount + o.discount_paid_by_company + o.shipping_amount - o.shipping_amount_no_discount) / ce.rate_eu ELSE 0 END) AS GFV_Gross_Log,
       SUM(CASE WHEN o.with_logistics = FALSE AND o.order_status = 'CONFIRMED' THEN (o.total_amount + o.shipping_amount + o.discount_paid_by_company) / ce.rate_eu ELSE 0 END) AS GMV_Gross_Mktp,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN (o.total_amount + o.discount_paid_by_company + o.shipping_amount - o.shipping_amount_no_discount) / ce.rate_eu ELSE 0 END) AS GFV_Gross_Total,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN o.shipping_amount_no_discount / ce.rate_eu ELSE 0 END) AS DF_No_Discount_Total,
       SUM(CASE WHEN o.with_logistics AND o.order_status = 'CONFIRMED' THEN o.shipping_amount_no_discount / ce.rate_eu ELSE 0 END) AS DF_No_Discount_Log,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN o.discount_paid_by_company / ce.rate_eu ELSE 0 END) AS Discounts_PEYA_Total
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,month) = ce.currency_exchange_date
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 1,2,3'''.format(lm,tm)

q_online = '''SELECT cn.country_name AS Country,
       FORMAT_DATE('%Y-%m',DATE(pm.full_date)) AS Month,
       p.business_type.business_type_name AS Business,
       COUNT(DISTINCT CASE WHEN pm.is_online THEN p.partner_id ELSE NULL END) AS Online,
       COUNT(DISTINCT CASE WHEN pm.is_churned THEN p.partner_id ELSE NULL END) AS Churns,
       COUNT(DISTINCT CASE WHEN pm.is_online AND pm.has_confirmed_orders = FALSE THEN p.partner_id ELSE NULL END) AS Zombies
FROM `peya-bi-tools-pro.il_core.fact_partners_monthly` AS pm
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON pm.restaurant_id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON p.country_id = cn.country_id
WHERE DATE(pm.full_date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND cn.country_name IS NOT NULL
GROUP BY 1,2,3'''.format(lm,tm)

q_freq = '''SELECT cn.country_name AS Country,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.user.id ELSE NULL END) AS Active_Users
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 1,2'''.format(lm,tm)

q_disc = '''SELECT cn.country_name AS Country,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       IFNULL(dt.discount_type_name,'-') AS Discount_Type,
       IFNULL(od.discount_paid_by,'-') AS Paid_By,
       COUNT(DISTINCT o.order_id) AS Confirmed,
       ROUND(IFNULL(SUM(od.discount_amount / ce.rate_eu),0),2) AS Amount
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
UNNEST (discounts) AS od
LEFT JOIN `peya-bi-tools-pro.il_core.dim_discount_type` AS dt ON od.discount_type_id = dt.discount_type_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,month) = ce.currency_exchange_date
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.order_status = 'CONFIRMED'
GROUP BY 1,2,3,4
ORDER BY 4,1,2,3,5 DESC'''.format(lm,tm)

In [8]:
# Descargo la data
hue_orders = pd.io.gbq.read_gbq(q_orders, project_id='peya-argentina', dialect='standard')

Downloading: 100%|████████████████████████████████████████████████████████████████| 695/695 [00:01<00:00, 436.75rows/s]


In [9]:
hue_online = pd.io.gbq.read_gbq(q_online, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████| 835/835 [00:00<00:00, 1719.17rows/s]


In [10]:
hue_freq = pd.io.gbq.read_gbq(q_freq, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 60.36rows/s]


In [11]:
hue_disc = pd.io.gbq.read_gbq(q_disc, project_id='peya-argentina', dialect='standard')

Downloading: 100%|████████████████████████████████████████████████████████████████| 570/570 [00:01<00:00, 508.60rows/s]


In [12]:
# Copio las bases
orders = hue_orders.copy()
online = hue_online.copy()
freq = hue_freq.copy()
disc = hue_disc.copy()

### TRABAJO

In [13]:
# Doy el mismo formato a todos los Months
orders['Month'] = pd.to_datetime(orders['Month'], format='%Y-%m').dt.strftime('%Y-%m')
online['Month'] = pd.to_datetime(online['Month'], format='%Y-%m').dt.strftime('%Y-%m')
freq['Month'] = pd.to_datetime(freq['Month'], format='%Y-%m').dt.strftime('%Y-%m')
disc['Month'] = pd.to_datetime(disc['Month'], format='%Y-%m').dt.strftime('%Y-%m')

In [14]:
# Hago un merge
final = orders.merge(online,on=['Country','Business','Month'],how='outer')
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [15]:
# Doy formato a las columnas
cols = [i for i in final.columns[3:]]
final[cols] = final[cols].astype(float)
cols = [i for i in freq.columns[2:]]
freq[cols] = freq[cols].astype(float)
cols = [i for i in disc.columns[4:]]
disc[cols] = disc[cols].astype(float)
# Elimino las filas con todos 0
final = final[(final[final.columns[3:]] != 0).any(axis=1)].copy()
freq = freq[(freq[freq.columns[2:]] != 0).any(axis=1)].copy()
disc = disc[(disc[disc.columns[4:]] != 0).any(axis=1)].copy()

In [16]:
# Separo Food/NV
final['Food/NV'] = final['Business'].apply(lambda x: 'Food' if x in ['Restaurant','Coffee'] else 'NV')
final['Restaurant/NV'] = final['Business'].apply(lambda x: 'Restaurant' if x == 'Restaurant' else 'NV')

In [39]:
total = disc[disc['Paid_By'] == 'COMPANY'].pivot_table(index=['Month'],columns=['Discount_Type'],values=['Amount'],aggfunc='sum',fill_value=0).reset_index()

In [45]:
total['Total'] = total['Amount']['BINS'] + total['Amount']['PARTNERSHIPS'] + total['Amount']['PLUS_SHIPPING_COST'] + total['Amount']['SUBSIDIZED'] + total['Amount']['VOUCHER']
total['%Voucher'] = round(total['Amount']['VOUCHER']/total['Total']*100,2)

In [46]:
total

Month     Amount                                             \
Discount_Type                BINS PARTNERSHIPS PLUS_SHIPPING_COST SUBSIDIZED   
0              2021-01  158040.80         0.00           58535.72  598506.51   
1              2021-02  149856.79        12.73           47022.30  832478.95   
2              2021-03  329006.42       244.20           50670.34  893519.60   
3              2021-04  267340.87      1218.62           45757.40  588241.11   
4              2021-05  279350.17      2807.06           37367.07  701017.74   
5              2021-06   17216.26        49.61            1790.12   44230.35   

                                Total %Voucher  
Discount_Type     VOUCHER                       
0              4537867.98  5352951.01    84.77  
1              4591044.21  5620414.98    81.69  
2              6744439.83  8017880.39    84.12  
3              7260856.71  8163414.71    88.94  
4              6833178.97  7853721.01    87.01  
5               361340.34   424626.68    85.10

### CARGA

In [190]:
# Carga Final
sheet_id = '1d1UWKgD0QTMe9_TnYRNaucyMYkv3R5EhQYBAM6gCIgY'
wks_name = 'Crudo'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(final, index=False, sheet=wks_name, replace=True)

In [163]:
# Carga Frecuencia
sheet_id = '1d1UWKgD0QTMe9_TnYRNaucyMYkv3R5EhQYBAM6gCIgY'
wks_name = 'Crudo Freq'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(freq, index=False, sheet=wks_name, replace=True)